# Visualization Techniques for AERMOD Results

This notebook demonstrates visualization methods for AERMOD concentration results.

## Topics Covered

1. Contour plots with matplotlib
2. Interactive maps with Folium
3. Multi-scenario comparisons
4. Wind rose and meteorology visualization
5. Publication-quality figures

**Prerequisites:**
- PyAERMOD installed
- AERMOD outputs available
- matplotlib, folium, numpy, pandas

## Setup

In [ ]:
from pyaermod.output_parser import parse_aermod_output, AERMODOutputParser
from pyaermod.visualization import AERMODVisualizer

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm
import warnings
warnings.filterwarnings('ignore')

# Optional: advanced visualization module
try:
    from pyaermod.advanced_viz import AdvancedVisualizer
    HAS_ADVANCED_VIZ = True
    print("✓ Advanced visualization available")
except ImportError:
    HAS_ADVANCED_VIZ = False
    print("ℹ Advanced visualization not available (scipy required)")

print("✓ Visualization modules imported")

## Example 1: Basic Contour Plot

Create a simple concentration contour plot.

In [ ]:
# Parse AERMOD output
parser = AERMODOutputParser()
output_file = "example_run.out"  # Replace with your file

try:
    results = parser.parse_output_file(output_file)
    
    if results['concentrations']:
        conc_df = results['concentrations'][0]  # First averaging period
        
        # Create contour plot
        viz = AERMODVisualizer(results)
        viz.plot_contours(
            df=conc_df,
            title="Ground-Level Concentration Contours",
            save_path="contour_basic.png"
        )
        
        print("✓ Created: contour_basic.png")
except FileNotFoundError:
    print(f"Note: {output_file} not found - run AERMOD first!")
    print("This example shows the code structure.")

## Example 2: Customized Contour Plot

Create publication-quality contour plot with custom styling.

In [ ]:
def create_publication_contour(df, title, pollutant="PM2.5", units="μg/m³"):
    """
    Create high-quality contour plot for publication.
    """
    # Extract coordinates and concentrations
    x = df['X'].values
    y = df['Y'].values
    conc = df['CONC'].values
    
    # Create regular grid
    x_unique = np.unique(x)
    y_unique = np.unique(y)
    X, Y = np.meshgrid(x_unique, y_unique)
    Z = conc.reshape(len(y_unique), len(x_unique))
    
    # Create figure
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Define contour levels (customize based on data)
    max_conc = np.max(conc)
    levels = np.logspace(np.log10(max_conc/1000), np.log10(max_conc), 15)
    
    # Create filled contours
    contourf = ax.contourf(X, Y, Z, levels=levels, 
                           cmap='YlOrRd', norm=mcolors.LogNorm())
    
    # Add contour lines
    contour = ax.contour(X, Y, Z, levels=levels, 
                         colors='black', linewidths=0.5, alpha=0.4)
    
    # Add colorbar
    cbar = plt.colorbar(contourf, ax=ax, pad=0.02)
    cbar.set_label(f'{pollutant} Concentration ({units})', 
                   fontsize=12, fontweight='bold')
    cbar.ax.tick_params(labelsize=10)
    
    # Mark source location
    ax.plot(0, 0, 'k^', markersize=15, markeredgewidth=2, 
            markerfacecolor='white', label='Source', zorder=5)
    
    # Styling
    ax.set_xlabel('Easting (m)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Northing (m)', fontsize=12, fontweight='bold')
    ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.legend(loc='upper right', fontsize=10, framealpha=0.9)
    
    plt.tight_layout()
    return fig, ax

# Example usage (with actual data):
# fig, ax = create_publication_contour(
#     df=conc_df,
#     title="Annual Average PM2.5 Concentrations",
#     pollutant="PM2.5",
#     units="μg/m³"
# )
# plt.savefig('publication_contour.png', dpi=300, bbox_inches='tight')
# plt.show()

print("✓ Publication contour function defined")

## Example 3: Interactive Folium Map

Create an interactive web map with concentration overlay.

In [ ]:
def create_interactive_map(df, center_lat, center_lon, title="Concentration Map"):
    """
    Create interactive Folium map with concentration overlay.
    
    Args:
        df: DataFrame with X, Y, CONC columns (in UTM coordinates)
        center_lat: Latitude of map center
        center_lon: Longitude of map center
        title: Map title
    """
    import folium
    from folium import plugins
    
    # Create base map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=13,
        tiles='OpenStreetMap'
    )
    
    # Add title
    title_html = f'''<h3 align="center" style="font-size:20px"><b>{title}</b></h3>'''
    m.get_root().html.add_child(folium.Element(title_html))
    
    # Note: In practice, you'd convert UTM (X, Y) to lat/lon
    # For this example, we show the structure
    
    # Add source marker
    folium.Marker(
        location=[center_lat, center_lon],
        popup='Emission Source',
        icon=folium.Icon(color='red', icon='industry', prefix='fa')
    ).add_to(m)
    
    # Add heat map layer (would need lat/lon conversion)
    # heat_data = [[lat, lon, conc] for lat, lon, conc in zip(lats, lons, concentrations)]
    # plugins.HeatMap(heat_data, radius=15, blur=25, max_zoom=13).add_to(m)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    return m

# Example usage:
# mapper = AERMODVisualizer(results)
# interactive_map = viz.create_interactive_map(
#     df=conc_df,
#     center_lat=40.7128,
#     center_lon=-74.0060,
#     title="PM2.5 Concentration Map"
# )
# interactive_map.save('interactive_map.html')

print("✓ Interactive map function defined")

## Example 4: Multi-Scenario Comparison

Compare concentration patterns from multiple scenarios side-by-side.

In [ ]:
def compare_scenarios(scenario_data, titles, overall_title="Scenario Comparison"):
    """
    Create side-by-side comparison of multiple scenarios.
    
    Args:
        scenario_data: List of DataFrames (one per scenario)
        titles: List of titles for each scenario
        overall_title: Overall figure title
    """
    n_scenarios = len(scenario_data)
    fig, axes = plt.subplots(1, n_scenarios, figsize=(6*n_scenarios, 5))
    
    if n_scenarios == 1:
        axes = [axes]
    
    # Find global min/max for consistent color scale
    all_conc = np.concatenate([df['CONC'].values for df in scenario_data])
    vmin, vmax = np.min(all_conc[all_conc > 0]), np.max(all_conc)
    
    for i, (df, title, ax) in enumerate(zip(scenario_data, titles, axes)):
        # Extract data
        x = df['X'].values
        y = df['Y'].values
        conc = df['CONC'].values
        
        # Create grid
        x_unique = np.unique(x)
        y_unique = np.unique(y)
        X, Y = np.meshgrid(x_unique, y_unique)
        Z = conc.reshape(len(y_unique), len(x_unique))
        
        # Plot
        im = ax.contourf(X, Y, Z, levels=20, cmap='YlOrRd', 
                        norm=mcolors.LogNorm(vmin=vmin, vmax=vmax))
        ax.contour(X, Y, Z, levels=10, colors='black', linewidths=0.3, alpha=0.3)
        
        # Mark source
        ax.plot(0, 0, 'k^', markersize=12, markeredgewidth=1.5, 
               markerfacecolor='white')
        
        # Styling
        ax.set_xlabel('Easting (m)', fontsize=10)
        ax.set_ylabel('Northing (m)', fontsize=10)
        ax.set_title(title, fontsize=12, fontweight='bold')
        ax.set_aspect('equal')
        ax.grid(True, alpha=0.2)
        
        # Add colorbar to last subplot
        if i == n_scenarios - 1:
            cbar = plt.colorbar(im, ax=axes, pad=0.02, aspect=30)
            cbar.set_label('Concentration (μg/m³)', fontsize=11, fontweight='bold')
    
    fig.suptitle(overall_title, fontsize=14, fontweight='bold', y=1.02)
    plt.tight_layout()
    
    return fig

# Example usage:
# scenario1_df = parser.parse_output_file('scenario1.out')['concentrations'][0]
# scenario2_df = parser.parse_output_file('scenario2.out')['concentrations'][0]
# scenario3_df = parser.parse_output_file('scenario3.out')['concentrations'][0]
# 
# fig = compare_scenarios(
#     scenario_data=[scenario1_df, scenario2_df, scenario3_df],
#     titles=["Baseline", "Reduced Emissions", "Taller Stack"],
#     overall_title="Control Strategy Comparison"
# )
# plt.savefig('scenario_comparison.png', dpi=300, bbox_inches='tight')

print("✓ Scenario comparison function defined")

## Example 5: Maximum Concentration Tracking

Track maximum concentration location across scenarios.

In [ ]:
def plot_max_concentration_trends(scenario_params, max_concentrations, 
                                   param_name="Parameter", param_units=""):
    """
    Plot how maximum concentration changes with parameter.
    
    Args:
        scenario_params: List of parameter values (e.g., stack heights)
        max_concentrations: List of maximum concentrations
        param_name: Name of varied parameter
        param_units: Units of parameter
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Left plot: Max concentration vs. parameter (linear)
    ax1.plot(scenario_params, max_concentrations, 
             marker='o', linewidth=2, markersize=8, color='steelblue')
    ax1.fill_between(scenario_params, 0, max_concentrations, alpha=0.3)
    ax1.set_xlabel(f'{param_name} ({param_units})', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Maximum Concentration (μg/m³)', fontsize=12, fontweight='bold')
    ax1.set_title('Linear Scale', fontsize=12, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # Right plot: Log scale
    ax2.semilogy(scenario_params, max_concentrations, 
                 marker='s', linewidth=2, markersize=8, color='coral')
    ax2.set_xlabel(f'{param_name} ({param_units})', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Maximum Concentration (μg/m³)', fontsize=12, fontweight='bold')
    ax2.set_title('Log Scale', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3, which='both')
    
    plt.tight_layout()
    return fig

# Example usage:
# stack_heights = [20, 30, 40, 50, 60, 70, 80]
# max_concs = [15.2, 8.7, 5.3, 3.4, 2.3, 1.7, 1.3]
# 
# fig = plot_max_concentration_trends(
#     scenario_params=stack_heights,
#     max_concentrations=max_concs,
#     param_name="Stack Height",
#     param_units="m"
# )
# plt.savefig('max_conc_trends.png', dpi=300)

print("✓ Trend plotting function defined")

## Example 6: Cross-Section Plots

Visualize concentration profiles along specific transects.

In [ ]:
def plot_cross_sections(df, source_x=0, source_y=0):
    """
    Plot concentration cross-sections through source location.
    
    Args:
        df: DataFrame with X, Y, CONC columns
        source_x: Source X coordinate
        source_y: Source Y coordinate
    """
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    
    # Extract E-W cross-section (through source, constant Y)
    ew_section = df[df['Y'] == source_y].sort_values('X')
    if not ew_section.empty:
        ax1.plot(ew_section['X'], ew_section['CONC'], 
                linewidth=2, marker='o', markersize=4, color='steelblue')
        ax1.axvline(source_x, color='red', linestyle='--', linewidth=2, label='Source')
        ax1.set_xlabel('Easting (m)', fontsize=11, fontweight='bold')
        ax1.set_ylabel('Concentration (μg/m³)', fontsize=11, fontweight='bold')
        ax1.set_title('East-West Cross-Section', fontsize=12, fontweight='bold')
        ax1.grid(True, alpha=0.3)
        ax1.legend()
    
    # Extract N-S cross-section (through source, constant X)
    ns_section = df[df['X'] == source_x].sort_values('Y')
    if not ns_section.empty:
        ax2.plot(ns_section['Y'], ns_section['CONC'], 
                linewidth=2, marker='s', markersize=4, color='coral')
        ax2.axvline(source_y, color='red', linestyle='--', linewidth=2, label='Source')
        ax2.set_xlabel('Northing (m)', fontsize=11, fontweight='bold')
        ax2.set_ylabel('Concentration (μg/m³)', fontsize=11, fontweight='bold')
        ax2.set_title('North-South Cross-Section', fontsize=12, fontweight='bold')
        ax2.grid(True, alpha=0.3)
        ax2.legend()
    
    plt.tight_layout()
    return fig

# Example usage:
# fig = plot_cross_sections(conc_df, source_x=0, source_y=0)
# plt.savefig('cross_sections.png', dpi=300)

print("✓ Cross-section plotting function defined")

## Example 7: Isopleths with Standard Overlay

Show concentration isopleths with air quality standard.

In [ ]:
def plot_with_standards(df, standards, pollutant="PM2.5", units="μg/m³"):
    """
    Create contour plot with air quality standards overlay.
    
    Args:
        df: DataFrame with X, Y, CONC
        standards: Dict of standard levels, e.g., {'NAAQS Annual': 12.0, 'NAAQS 24-hr': 35.0}
        pollutant: Pollutant name
        units: Concentration units
    """
    # Extract data
    x = df['X'].values
    y = df['Y'].values
    conc = df['CONC'].values
    
    # Create grid
    x_unique = np.unique(x)
    y_unique = np.unique(y)
    X, Y = np.meshgrid(x_unique, y_unique)
    Z = conc.reshape(len(y_unique), len(x_unique))
    
    # Create figure
    fig, ax = plt.subplots(figsize=(12, 10))
    
    # Define contour levels including standards
    std_values = sorted(standards.values())
    max_conc = np.max(conc)
    levels = np.concatenate([
        np.linspace(0, std_values[0], 5),
        std_values,
        np.linspace(std_values[-1], max_conc, 5)
    ])
    levels = np.unique(levels)
    
    # Create filled contours
    contourf = ax.contourf(X, Y, Z, levels=levels, cmap='YlOrRd', extend='max')
    
    # Highlight standard contours
    for name, value in standards.items():
        cs = ax.contour(X, Y, Z, levels=[value], colors='blue', linewidths=3)
        ax.clabel(cs, inline=True, fontsize=10, fmt=f'{name}\n{value:.1f}')
    
    # Mark source
    ax.plot(0, 0, 'k^', markersize=15, markeredgewidth=2, 
           markerfacecolor='white', label='Source', zorder=5)
    
    # Colorbar
    cbar = plt.colorbar(contourf, ax=ax, pad=0.02)
    cbar.set_label(f'{pollutant} ({units})', fontsize=12, fontweight='bold')
    
    # Styling
    ax.set_xlabel('Easting (m)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Northing (m)', fontsize=12, fontweight='bold')
    ax.set_title(f'{pollutant} Concentrations with Air Quality Standards', 
                fontsize=14, fontweight='bold', pad=20)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.legend(loc='upper right', fontsize=10)
    
    plt.tight_layout()
    return fig

# Example usage:
# standards = {
#     'NAAQS Annual': 12.0,
#     'NAAQS 24-hr': 35.0
# }
# fig = plot_with_standards(conc_df, standards, pollutant="PM2.5")
# plt.savefig('concentrations_with_standards.png', dpi=300, bbox_inches='tight')

print("✓ Standards overlay plotting function defined")

## Example 8: Polar Concentration Plot

Visualize concentrations in polar coordinates (useful for polar grids).

In [ ]:
def plot_polar_concentrations(df):
    """
    Create polar plot of concentrations.
    Useful when receptor grid is polar.
    """
    # Convert X, Y to polar coordinates
    df['distance'] = np.sqrt(df['X']**2 + df['Y']**2)
    df['angle'] = np.arctan2(df['Y'], df['X'])
    
    # Create polar plot
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='polar')
    
    # Create scatter plot
    scatter = ax.scatter(df['angle'], df['distance'], 
                        c=df['CONC'], cmap='YlOrRd', 
                        s=50, alpha=0.7, edgecolors='black', linewidths=0.5)
    
    # Colorbar
    cbar = plt.colorbar(scatter, ax=ax, pad=0.1)
    cbar.set_label('Concentration (μg/m³)', fontsize=11, fontweight='bold')
    
    # Styling
    ax.set_theta_zero_location('N')
    ax.set_theta_direction(-1)  # Clockwise
    ax.set_title('Concentration Distribution (Polar View)', 
                fontsize=14, fontweight='bold', pad=20)
    
    return fig

# Example usage:
# fig = plot_polar_concentrations(conc_df)
# plt.savefig('polar_concentrations.png', dpi=300)

print("✓ Polar plotting function defined")

## Summary

This notebook demonstrated:

1. **Basic contours** - Simple concentration visualization
2. **Publication plots** - High-quality figures with custom styling
3. **Interactive maps** - Web-based Folium maps
4. **Scenario comparison** - Side-by-side analysis
5. **Trend plots** - Parameter sensitivity visualization
6. **Cross-sections** - Concentration profiles
7. **Standards overlay** - Compliance assessment
8. **Polar plots** - Alternative visualization approach

## Advanced Visualization

PyAERMOD's `AdvancedVisualizer` module provides additional capabilities:

### 3D Surface Plots

```python
from pyaermod.advanced_viz import AdvancedVisualizer

adv_viz = AdvancedVisualizer(results)

# 3D surface plot of concentrations
fig = adv_viz.plot_3d_surface(
    averaging_period="ANNUAL",
    title="3D Concentration Surface",
    colormap="YlOrRd",
)
plt.show()
```

### Time-Series Animation

For POSTFILE data with multiple timesteps, create animated GIFs:

```python
from pyaermod.postfile import read_postfile

result = read_postfile("postfile.bin")
frames = result.data  # DataFrame with timestep, receptor, concentration columns

# Create animation frames
fig = adv_viz.plot_time_series_animation(
    frames=frames,
    output_path="concentration_animation.gif",
    fps=4,
)
```

### Exceedance Probability Maps

```python
# Plot probability of exceeding a threshold at each receptor
fig = adv_viz.plot_exceedance_probability(
    threshold=35.0,  # ug/m3 (e.g., PM2.5 24-hr NAAQS)
    averaging_period="24",
    title="Exceedance Probability Map",
)
```

### Next Notebooks

- **Notebook 06**: POSTFILE Analysis — parse binary/text output, timestep queries, deposition data
- **Notebook 07**: Advanced Features — background concentrations, deposition, EVENT processing, NO2 chemistry

### Visualization Best Practices

1. **Choose appropriate scale**
   - Linear: When concentrations span < 1 order of magnitude
   - Log: When concentrations span > 2 orders of magnitude

2. **Color schemes**
   - Sequential (YlOrRd, Blues): Single variable
   - Diverging (RdBu): Differences/changes
   - Consider colorblind-friendly palettes

3. **Consistency**
   - Use same color scale for comparing scenarios
   - Maintain aspect ratio for spatial data
   - Include scale bars and legends

4. **Publication quality**
   - DPI >= 300 for print
   - Vector formats (PDF, SVG) for scalability
   - Clear labels with appropriate font sizes

| Tool | Best For |
|------|----------|
| **matplotlib** | Static plots, publications, reports |
| **Folium** | Interactive web maps, stakeholder engagement |
| **AdvancedVisualizer** | 3D surfaces, animations, exceedance maps |